In [1]:
import torch
from utils.experimentorUtils import getExperimentor
from utils.constants import *

In [2]:
config = {
    "dataset_name": Dataset.CORA,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 500,
    'num_of_runs': 3,
    'patience_period': 100,
    
    'batch_size': 32,
    'test_batch_size': 32,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 5,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.005,
    'num_of_layers': 2, 
    'num_heads': 1,
    'hidden_size': 128,
    'dropout': 0.6,  
    "use_layer_norm": False,
    "use_batch_norm": False,
    
    'nbor_degree': 1,
    'adj_mode': AdjacencyMode.OneStep,
    'sparse': False
}

In [3]:
experimentor = getExperimentor(config["dataset_name"])(config)
experimentor.run_wrapper()

Run 01:


Epoch 01: 100%|██████████| 2708/2708 [00:04<00:00, 639.61it/s] 


Epoch 01| Loss: 0.8081| Acc: 0.7607| Train Time: 4.2389s
Epoch 02| Loss: 0.4240| Acc: 0.8848| Train Time: 3.6064s
Epoch 03| Loss: 0.3065| Acc: 0.9021| Train Time: 3.5260s
Epoch 04| Loss: 0.2349| Acc: 0.9280| Train Time: 3.8825s
Epoch 05| Loss: 0.2025| Acc: 0.9335| Train Time: 3.7362s


Evaluating: 100%|██████████| 5416/5416 [00:05<00:00, 933.53it/s] 


Train: 0.9786| Val: 0.9940| Test: 0.9850| Eval Time: 5.8056s
Epoch 06| Loss: 0.1545| Acc: 0.9498| Train Time: 3.9860s
Epoch 07| Loss: 0.1696| Acc: 0.9505| Train Time: 3.7705s
Epoch 08| Loss: 0.1209| Acc: 0.9620| Train Time: 4.1658s
Epoch 09| Loss: 0.1207| Acc: 0.9623| Train Time: 3.8180s
Epoch 10| Loss: 0.1310| Acc: 0.9579| Train Time: 4.1076s


Evaluating: 100%|██████████| 5416/5416 [00:06<00:00, 852.16it/s]


Train: 0.9929| Val: 0.9960| Test: 0.9950| Eval Time: 6.3617s
Epoch 11| Loss: 0.1277| Acc: 0.9609| Train Time: 3.9031s
Epoch 12| Loss: 0.1197| Acc: 0.9686| Train Time: 4.0904s
Epoch 13| Loss: 0.1113| Acc: 0.9634| Train Time: 3.8277s
Epoch 14| Loss: 0.0969| Acc: 0.9697| Train Time: 3.7624s
Epoch 15| Loss: 0.0981| Acc: 0.9705| Train Time: 3.7623s


Evaluating: 100%|██████████| 5416/5416 [00:05<00:00, 906.71it/s] 


Train: 1.0000| Val: 1.0000| Test: 0.9980| Eval Time: 5.9772s
Epoch 16| Loss: 0.0945| Acc: 0.9719| Train Time: 3.8123s
Epoch 17| Loss: 0.1060| Acc: 0.9701| Train Time: 3.9720s
Epoch 18| Loss: 0.1305| Acc: 0.9653| Train Time: 3.8482s
Epoch 19| Loss: 0.1086| Acc: 0.9694| Train Time: 4.4207s
